<a href="https://colab.research.google.com/github/enVives/TFG/blob/main/FeatureDistances.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import torch
import cv2
import csv
import random
import numpy as np
import os
import time
import matplotlib.pyplot as plt
import kagglehub
import pandas as pd
import wandb
import pylab as pl
import json

from glob import glob
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score
from sklearn.model_selection import train_test_split
from torchvision import transforms,models
from torch import nn
from torch.utils.data import Dataset
from google.colab import files
from IPython.display import clear_output,display
from google.colab.patches import cv2_imshow
from PIL import Image
from skimage import io
from google.colab import drive


DOWNLOAD = False
CLAHE = False
SEGMENTATION = True
# ORIGEN = '/content/drive/MyDrive/HAM10000/skin-cancer-mnist-ham10000/'

drive.mount('/content/drive')


if DOWNLOAD:
  !rm -rf /content/sample_data/*

  !kaggle datasets download -d "kmader/skin-cancer-mnist-ham10000"

  !unzip -o skin-cancer-mnist-ham10000.zip -d /content/sample_data/

  !kaggle datasets download -d "tschandl/ham10000-lesion-segmentations"

  !unzip -o ham10000-lesion-segmentations.zip -d /content/sample_data/

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
# !mkdir -p ~/.kaggle
# !mv /content/kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json


In [14]:
metadates = pd.read_csv('/content/sample_data/HAM10000_metadata.csv')
metadates = metadates.sort_values(by='image_id')

In [15]:
class Formes(Dataset):
  #Classe on gestionarem les imatges dels fitxers
  dict_illnesses = {0 : 'nv', 1 : 'mel', 2 : 'bkl', 3 : 'bcc', 4 : 'akiec', 5 : 'vasc', 6 : 'df'}

  def __init__(self, images, labels, transform):
        super().__init__()
        self.paths = images
        self.labels = labels
        self.len = len(self.paths)
        self.transform = transform
        #Per defecte pens que el color pot extreure característiques importants, per tant en primer lloc
        #entrenarem les imatges de color
        self.greyscale = False

  def __len__(self):
        return self.len

  def __setmasks__(self,masks):
      self.masks = masks

  def __getmask__(self,i):
      return self.masks[i]

  def __lenmasks__(self):
    return len(self.masks)

  def __addlabel__(self,label):
    self.labels = np.append(self.labels,label)

  def __addPath__(self,path):
    self.paths = np.append(self.paths,path)

  def __getdist__(self):
    return pd.Series(self.labels).value_counts()

  def __getlabel__(self,i):
    return self.labels[i]

  def __getlabels__(self):
    classes = [[],[],[],[],[],[],[]]
    for i in range(len(self.labels)):
      classes[self.labels[i]].append(i) #afegim l'index
    return classes

  def __redistribute__(self,percentages):
    #percentages: [15,15,10,10,5,5] percentatges que volem pujar de la resta de classes llevat de nv
    threshold = 0.005  #percentatge de marge que deixam a la redistribució
    Ntarget = self.len
    classes = self.__getlabels__() #indexos de cada clase
    afegir = np.array([0,0,0,0,0,0],dtype=np.int64) # de nv mai haurem d'afegir

    nmel = len(classes[1]) #nombre inicial de cada clase
    nbkl = len(classes[2])
    nbcc = len(classes[3])
    nakiec = len(classes[4])
    nvasc = len(classes[5])
    ndf = len(classes[6])

    while True:

      suma_actual = afegir.sum()

      operacio = percentages[0]*Ntarget - nmel
      afegir[0] +=  operacio if operacio > 0 else 0
      nmel += operacio if operacio > 0 else 0

      operacio = percentages[1]*Ntarget - nbkl
      afegir[1] += operacio if operacio > 0 else 0
      nbkl += operacio if operacio > 0 else 0

      operacio = percentages[2]*Ntarget - nbcc
      afegir[2] += operacio if operacio > 0 else 0
      nbcc += operacio if operacio > 0 else 0

      operacio = percentages[3]*Ntarget - nakiec
      afegir[3] += operacio if operacio > 0 else 0
      nakiec += operacio if operacio > 0 else 0

      operacio = percentages[4]*Ntarget - nvasc
      afegir[4] += operacio if operacio > 0 else 0
      nvasc += operacio if operacio > 0 else 0

      operacio = percentages[5]*Ntarget - ndf
      afegir[5] += operacio if operacio > 0 else 0
      ndf += operacio if operacio > 0 else 0

      if (afegir.sum()-suma_actual) < Ntarget*threshold:
        break

      Ntarget += (afegir.sum()-suma_actual)

    #Quedaria afegir a les imatges les còpies
    for i in range(len(afegir)):
      for j in range(afegir[i]):

          self.__addPath__(self.paths[classes[i+1][random.randint(0, len(classes[i+1]) - 1)]])
          self.__addlabel__(i+1)

    self.len = len(self.labels)

  def __setgreyscale__(self,mode):
    self.greyscale = mode

  def __getpath__(self,index):
    return self.paths[index]

  def __getitem__(self, index):
      path = self.paths[index]
      label = self.labels[index]

      image = cv2.imread(path, cv2.IMREAD_GRAYSCALE if self.greyscale else cv2.IMREAD_COLOR)

      if SEGMENTATION:
        mask = cv2.imread(self.masks[index], cv2.IMREAD_GRAYSCALE)
        _, mask_binaria = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
        image = cv2.bitwise_and(image, image, mask=mask_binaria)

        non_black_pixels = np.where(image > 0)

        x_min, x_max = np.min(non_black_pixels[1]), np.max(non_black_pixels[1])
        y_min, y_max = np.min(non_black_pixels[0]), np.max(non_black_pixels[0])

        marge = 10
        if ((x_min - marge) >= 0):
            x_min = x_min - marge
        if ((x_max + marge) <= 224):
            x_max = x_max + marge

        cropped_image = image[y_min:y_max, x_min:x_max]
        image = cv2.resize(cropped_image, (224, 224), interpolation=cv2.INTER_CUBIC)

      #preprocessament
      if CLAHE:
        lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
        l, a, b = cv2.split(lab)
        clahe = cv2.createCLAHE(clipLimit=1.5, tileGridSize=(8,8))
        l_clahe = clahe.apply(l)
        lab_clahe = cv2.merge((l_clahe, a, b))
        image = cv2.cvtColor(lab_clahe, cv2.COLOR_LAB2BGR)




      if self.greyscale:
        image = Image.fromarray(image, mode="L")
      else:
        image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

      image = self.transform(image)

      return image, label

In [16]:
def load_sets(transformation_training,transformation_default,training_dist,valitation_dist,testing_dist,distribution):

  illnes_dictionary = {
      'nv': 'Melanocytic nevi',
      'mel': 'Melanoma',
      'bkl': 'Benign keratosis-like lesions ',
      'bcc': 'Basal cell carcinoma',
      'akiec': 'Actinic keratoses and intraepithelial carcinoma / Bowens disease',
      'vasc': 'Vascular lesions',
      'df': 'Dermatofibroma'
  }

  img_files_1 = sorted(glob('/content/sample_data/HAM10000_images_part_1/*'))
  img_files_2 = sorted(glob('/content/sample_data/HAM10000_images_part_2/*'))
  img_files = img_files_1 + img_files_2

  img_files = np.array(img_files)

  mask_files = sorted(glob('/content/sample_data/HAM10000_segmentations_lesion_tschandl/*'))

  imgid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x for x in img_files}

  mask_path_dict = {os.path.splitext(os.path.basename(x))[0].replace('_segmentation', ''): x for x in mask_files}

  #print(mask_path_dict)

  #{'ISIC_0024306_segmentation': '/content/sample_data/HAM10000_segmentations_lesion_tschandl/ISIC_0024306_segmentation.png'

  metadates['path'] = metadates['image_id'].map(imgid_path_dict.get)
  metadates['mask_path'] = metadates['image_id'].map(mask_path_dict.get)

  metadates['illness'] = metadates['dx'].map(illnes_dictionary.get)
  metadates['illness_code'] = metadates['dx'].map({'nv': 0, 'mel': 1, 'bkl': 2, 'bcc': 3, 'akiec': 4, 'vasc': 5, 'df': 6})

  #Aquest illness_code s'utilitzarà com a label de la enfermetat

  img_number = len(img_files)

  X = metadates.drop('illness_code',axis= 1)
  y = metadates['illness_code']

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testing_dist, random_state=42, stratify=y)
  X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=valitation_dist/(training_dist+valitation_dist), random_state=42, stratify=y_train)

  #Una bona idea seria aplicar data augmentation al conjunt d'entrenament ja que les classes estan molt desbalancejades
  #Una altra bona idea seria emplear una funció de pèrdua que tengui en compte les classes desbalancejades.

  #Antes de guardar els datasets en classes Formes, hauriem de caluclar la mitjana i desviació típica de les imatges
  train_data = Formes(X_train['path'].to_numpy(),y_train.to_numpy(),transformation_training)
  test_data = Formes(X_test['path'].to_numpy(),y_test.to_numpy(),transformation_default)
  validation_data = Formes(X_val['path'].to_numpy(),y_val.to_numpy(),transformation_default)

  if SEGMENTATION:

    train_data.__setmasks__(X_train['mask_path'].to_numpy())
    test_data.__setmasks__(X_test['mask_path'].to_numpy())
    validation_data.__setmasks__(X_val['mask_path'].to_numpy())

  #prova1: [0.12,0.12,0.06,0.04,0.02,0.02]
  #prova2: [0.12,0.12,0.06,0.04,0.02,0.015]
  #prova3: [0.13,0.13,0.07,0.05,0.02,0.015]

  if distribution != None:
    train_data.__redistribute__(distribution)

  return train_data, validation_data,test_data

In [17]:
from io import TextIOWrapper
from numpy.linalg import norm
import sys
import ast
import shutil

def genera_dists(model,test_data,activation):

  PATH = '/content/drive/MyDrive/Features/Embeddings/embeddings_pesos_resnet152multiclass_5.pt.txt'

  csv.field_size_limit(sys.maxsize)
  dict_illnesses = {0 : 'nv', 1 : 'mel', 2 : 'bkl', 3 : 'bcc', 4 : 'akiec', 5 : 'vasc', 6 : 'df'}

  total = test_data.__len__()
  DISTANCE = 1

  data = [["test_img","test_img_label","top_distances","top_labels","bottom_distances","bottom_labels"]]

  ROOT = '/content/drive/MyDrive/Features/Dist/'

  current_runs = len([d for d in os.listdir(ROOT) if os.path.isdir(os.path.join(ROOT, d))])
  os.makedirs(ROOT+str(current_runs),exist_ok=True)


  f = ROOT+str(current_runs)+'/distances.txt'

  headers = ["test_img","test_img_pred_label","test_img_label","top_distances","top_images_labels","top_images_names","bottom_distances","bottom_images_labels","bottom_images_names"]

  with open(f, mode="w", newline="") as file:
      writer = csv.writer(file, delimiter=';')
      writer.writerow(headers)  # Write header

  #iteram imatges de test
  for i in range(total):

    imatge,pred_label = test_data.__getitem__(i)
    actual_label = test_data.__getlabel__(i)

    pred_label = dict_illnesses[pred_label]
    actual_label = dict_illnesses[actual_label]

    imatge = imatge.to(device)
    imatge = imatge.unsqueeze(0)

    img_path = test_data.__getpath__(i)
    img_name = img_path.split('/')
    img_name = img_name[len(img_name)-1]

    zeros = [0,0,0,0,0]
    inf = [np.inf,np.inf,np.inf,np.inf,np.inf]
    if DISTANCE == 0:
      menor = np.inf
      top = np.array(zeros,dtype=object)
      bottom = np.array(inf,dtype=object)
    else:
      menor = 0
      top = np.array(inf,dtype=object)
      bottom = np.array(zeros,dtype=object)

    classes_properes_top = np.array([-1,-1,-1,-1,-1],dtype=object)
    classes_properes_bottom = np.array([-1,-1,-1,-1,-1],dtype=object)

    paths_top = np.array(["","","","",""],dtype=object)
    paths_bottom = np.array(["","","","",""],dtype=object)
    names_top = np.array(["","","","",""],dtype=object)
    names_bottom = np.array(["","","","",""],dtype=object)

    output = model(imatge)

    my_feature = activation['lastlayer'].flatten().cpu().detach().numpy()

    with open(PATH, mode='r') as file:

      features = csv.reader(file,delimiter=',')
      #botam la capçalera
      next(features)

      for rows in features:

        fcnoutput = np.array(ast.literal_eval(rows[1]))
        path = rows[2]
        label = rows[3]

        comparing_feature = fcnoutput
        if DISTANCE == 0:
          distance = np.dot(my_feature,comparing_feature)/(norm(my_feature)*norm(comparing_feature))
        else:
          distance = norm(my_feature - comparing_feature)


        #falta corregir pel canvi de calcul de distàncies
        if DISTANCE == 0:
          for j in range(len(top)):
            if distance < menor:
              menor = distance
            if distance > top[j]:
              top[j] = distance
              classes_properes_top[j] = dict_illnesses[int(label)]
              paths_top[j] = path
              names_top[j] = path.split("/")[-1].split(".")[0]
              break
            if distance < bottom[j]:
              bottom[j] = distance
              classes_properes_bottom[j] = dict_illnesses[int(label)]
              paths_bottom[j] = path
              names_bottom[j] = path.split("/")[-1].split(".")[0]
              break
        else:
          for j in range(len(top)):
            if distance > menor:
              menor = distance
            if distance < top[j]:
              top[j] = distance
              classes_properes_top[j] = dict_illnesses[int(label)]
              paths_top[j] = path
              names_top[j] = path.split("/")[-1].split(".")[0]
              break
            if distance > bottom[j]:
              bottom[j] = distance
              classes_properes_bottom[j] = dict_illnesses[int(label)]
              paths_bottom[j] = path
              names_bottom[j] = path.split("/")[-1].split(".")[0]
              break


    #carpeta de la nostra imatge
    os.makedirs(ROOT+str(current_runs)+'/'+img_name.split('.')[0]+'('+actual_label+')/',exist_ok=True)


    if DISTANCE == 0:
      sorted_indices = np.argsort(top)[::-1]
    else:
      sorted_indices = np.argsort(top)

    #si utilitzam la distància cosin ens interessa que la distància més gran sigui la primera

    top = top[sorted_indices]
    classes_properes_top = classes_properes_top[sorted_indices]
    paths_top = paths_top[sorted_indices]
    names_top = names_top[sorted_indices]

    if DISTANCE == 0:
      sorted_indices = np.argsort(bottom)[::-1]
    else:
      sorted_indices = np.argsort(bottom)

    bottom = bottom[sorted_indices]
    classes_properes_bottom = classes_properes_bottom[sorted_indices]
    paths_bottom = paths_bottom[sorted_indices]
    names_bottom = names_bottom[sorted_indices]

    data_row = [img_name.split('.')[0],pred_label,actual_label,top.astype(float).tolist(),classes_properes_top,names_top,bottom.astype(float).tolist(),classes_properes_bottom,names_bottom]

    #a partir d'aqui tendrem les distàncies guardades de menys a mes

    os.makedirs('/content/drive/MyDrive/Features/Dist/'+str(current_runs)+'/'+img_name.split('.')[0]+'('+actual_label+')'+'/Top/',exist_ok=True)
    os.makedirs('/content/drive/MyDrive/Features/Dist/'+str(current_runs)+'/'+img_name.split('.')[0]+'('+actual_label+')'+'/Bottom/',exist_ok=True)

    for k in range(len(paths_top)):
      shutil.copy(paths_top[k], '/content/drive/MyDrive/Features/Dist/'+str(current_runs)+'/'+img_name.split('.')[0]+'('+actual_label+')/Top/'+classes_properes_top[k]+'_'+str(k)+'.jpg')

    shutil.copy(img_path,'/content/drive/MyDrive/Features/Dist/'+str(current_runs)+'/'+img_name.split('.')[0]+'('+actual_label+')/'+actual_label+'_imatge_test_'+img_name)

    for l in range(len(paths_bottom)):
      #print('/content/drive/MyDrive/Features/Dist/'+str(current_runs)+'/'+img_name.split('.')[0]+'('+actual_label+')/Bottom/'+classes_properes_bottom[l]+'_'+str(l)+'.jpg')
      shutil.copy(paths_bottom[l], '/content/drive/MyDrive/Features/Dist/'+str(current_runs)+'/'+img_name.split('.')[0]+'('+actual_label+')/Bottom/'+classes_properes_bottom[l]+'_'+str(l)+'.jpg')

    with open(f, mode="a", newline="") as file:
          writer = csv.writer(file, delimiter=';')
          writer.writerow(data_row)  # Write row immediately

    print(i)
    if i == 5:
      break

  print("CSV file written successfully.")
  print(menor)

In [18]:
mean = torch.tensor([194.57463374, 139.13953272, 145.36132088]) /255 #rgb
std = torch.tensor([35.92275236, 38.90347617, 43.33101831]) / 255

TRAINING = 0.80
VALIDATION = 0.10
TESTING = 0.10
SIZE = 224

DISTRIBUTIONS = None

transform = transforms.Compose([
    transforms.Resize((SIZE,SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean = mean, std=std)
])

transform_training = transforms.Compose([
    transforms.Resize((SIZE,SIZE)),
    transforms.RandomHorizontalFlip(p=0.3),
    transforms.RandomVerticalFlip(p=0.3),
    transforms.RandomRotation(degrees=20),
    #transforms.RandomResizedCrop(224, scale=(0.8, 1.1)),
    transforms.ToTensor(),
    transforms.Normalize(mean = mean, std=std),
])

CLAHE = False
train_data,validation_data,test_data = load_sets(transform_training,transform,TRAINING,VALIDATION,TESTING,DISTRIBUTIONS)

PATH = '/content/drive/MyDrive/Runs/Ham10000Resnet152Multiclass/5/pesos_resnet152multiclass_5.pt'

resnetmulticlass152 = models.resnet152()
num_features = resnetmulticlass152.fc.in_features
resnetmulticlass152.fc = nn.Linear(in_features=num_features, out_features=7)

resnetmulticlass152.load_state_dict(torch.load(PATH, weights_only=True))

activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = input[0].detach()
    return hook


model = resnetmulticlass152
#model.layer4.register_forward_hook(get_activation('layer4'))

model.fc.register_forward_hook(get_activation('lastlayer'))

model.to(device)

genera_dists(model,test_data,activation)

0
1
2
3
4
5
CSV file written successfully.
2.8084382038625404
